In [1]:
using ModelingToolkit, Plots, Revise, Unitful, Symbolics, Logging, XSteam, Printf
using Graphs, Plots, GraphRecipes
using DifferentialEquations,  OrdinaryDiffEq

In [3]:
# Basic electric components
@variables t
const D = Differential(t)

@connector function HydraulicPin(; name)
    @variables P(t)=1.0 Q(t)=1.0
    ODESystem(Equation[], t, [v, i], [], name = name)
end

function Ground(; name)
    @named g = HydraulicPin()
    eqs = [g.P ~ 0]
    compose(ODESystem(eqs, t, [], [], name = name), g)
end

function ConstantPressure(; name, P = 1.0)
    val = V
    @named p = HydraulicPin()
    @named n = HydraulicPin()
    @parameters V = V
    eqs = [P ~ p.P - n.P
           0 ~ p.Q + n.Q]
    compose(ODESystem(eqs, t, [], [V], name = name), p, n)
end

function FlowCapacitor(; name, rho = 8050, V = 1, cp = 460, TAmbient = 293.15)
    @parameters rho=rho V=V cp=cp
    C = rho * V * cp
    @named p = HydraulicPin()
    @named n = HydraulicPin()
    eqs = [
        D(h.T) ~ h.Q_flow / C,
    ]
    compose(ODESystem(eqs, t, [], [rho, V, cp],
                      name = name), h)
end

function Capacitor(; name, C = 1.0)
    @named p = Pin()
    @named n = Pin()
    @variables v(t) = 0.0
    @parameters C = C
    eqs = [v ~ p.v - n.v
           0 ~ p.i + n.i
           D(v) ~ p.i / C]
    compose(ODESystem(eqs, t, [v], [C],
                      name = name), p, n)
end

function parallel_rc_model(i; name, source, ground, R, C)
    resistor = HeatingResistor(name = Symbol(:resistor, i), R = R)
    capacitor = Capacitor(name = Symbol(:capacitor, i), C = C)
    heat_capacitor = HeatCapacitor(name = Symbol(:heat_capacitor, i))

    rc_eqs = [connect(source.p, resistor.p)
              connect(resistor.n, capacitor.p)
              connect(capacitor.n, source.n, ground.g)
              connect(resistor.h, heat_capacitor.h)]

    compose(ODESystem(rc_eqs, t, name = Symbol(name, i)),
            [resistor, capacitor, source, ground, heat_capacitor])
end



1-element Vector{Equation}:
 Differential(t)(Bsys) ~ Ḃout + Differential(t)(Bcv) - Ḃin